The Biggest weakness in the competitive analysis was (the 2.6% comparability) and then you solve it with a more sophisticated data science technique. We will solve this issue

The Goal: Move the exact name matching to cononical product matching. We want to group "Coca-Cola 500ml", "Coke Classic 500ml Bottle", and "Coca-Cola Original Taste 0.5L" into a single canonical product: "coca-cola-500ml". This will massively expand our comparable product universe.

## e5-large transformer to Create Canonical Products

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from thefuzz import process, fuzz
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.data_processing import normalise_product_name

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

In [5]:
INTERIM_PATH = "C:/Project/UK store analysis/data/01_interim/cleaned_supermarket_data.parquet"
df_comp = pd.read_parquet(INTERIM_PATH)

print("Cleaned data loaded successfully")
df_comp.head(3)

Cleaned data loaded successfully


,supermarket,prices,prices_unit,unit,names,date,category,own_brand
0,Aldi,3.09,0.14,unit,Mamia Ultra-fit Peppa Pig Nappy Pants 22 Pack/...,2024-04-13,baby_products,False
1,Aldi,3.09,0.17,unit,Mamia Ultra-fit Peppa Pig Nappy Pants 18 Pack/...,2024-04-13,baby_products,False
2,Aldi,3.59,0.09,unit,Mamia Ultra-fit Nappy Pants 40 Pack/Size 4,2024-04-13,baby_products,False


In [6]:
df_comp['normalized_name'] = df_comp['names'].apply(normalise_product_name)

# Check a few results
df_comp[['names', 'normalized_name']].head(10)


,names,normalized_name
0,Mamia Ultra-fit Peppa Pig Nappy Pants 22 Pack/...,mamia ultrafit peppa pig nappy pants size 6
1,Mamia Ultra-fit Peppa Pig Nappy Pants 18 Pack/...,mamia ultrafit peppa pig nappy pants size 7
2,Mamia Ultra-fit Nappy Pants 40 Pack/Size 4,mamia ultrafit nappy pants size 4
3,Mamia Boy's Night Pants 15 Pack,mamia boys night pants
4,Mamia Girl's Night Pants 15 Pack,mamia girls night pants
5,Mamia Newborn Nappies 24 Pack/Size 1 2-5kg/4-1...,mamia newborn nappies size 1 2411lbs
6,Mamia Nappies Ultra Dry Air System 22 Pack/Siz...,mamia nappies ultra dry air system size 7 xxl
7,Mamia Newborn Nappies 56 Pack/Size 3 Midi 4-9k...,mamia newborn nappies size 3 midi 4920lbs
8,Mamia Newborn Nappies 60 Pack/Size 2 Mini 3-6k...,mamia newborn nappies size 2 mini 3613lbs
9,Mamia Ultra-fit Nappy Pants 28 Pack/Size 7,mamia ultrafit nappy pants size 7


In [7]:
# Extract unique normalised names
unique_names = df_comp["normalized_name"].dropna().unique()

print(f"Unique product names: {len(unique_names)}")

Unique product names: 116229


In [8]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("intfloat/e5-large", device=device)
# Format inputs with "query: " prefix (as per e5 training protocol)
formatted_names = ["query: " + name for name in unique_names]

print(device)
print(model)

KeyboardInterrupt: 

In [ ]:
# Compute embedding in batches
embeddings = model.encode(
    formatted_names,
    batch_size=128,
    show_progress_bar=True,
    convert_to_numpy=True
)

Batches:   0%|          | 0/909 [00:00<?, ?it/s]

In [ ]:
import faiss

# Normalise for cosine similarity
faiss.normalize_L2(embeddings)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

k = 5
distances, indices = index.search(embeddings, k)

In [ ]:
# Canonical mapping
similarity_threshold = 0.85
mapping = {}

for i, name in enumerate(unique_names):
    similar_indices = [idx for idx, dist in zip(indices[i], distances[i]) if dist >= similarity_threshold]
    for idx in similar_indices:
        neighbour_name = unique_names[idx]
        canonical = min(name, neighbour_name)
        mapping[neighbour_name] = canonical

# Apply the mapping
df_comp["canonical_name"] = df_comp["normalized_name"].map(mapping)
df_comp["canonical_name"] = df_comp["canonical_name"].fillna(df_comp["normalized_name"])

In [ ]:
OUTPUT_PATH = "C:/Project/UK store analysis/data/02_processed/canonical_products_e5.parquet"
df_comp.to_parquet(OUTPUT_PATH)

print(" Canonical product mapping complete and saved.")


In [ ]:
df_comp.head(10)

,supermarket,prices,prices_unit,unit,names,date,category,own_brand,normalized_name,canonical_name
0,Aldi,3.09,0.14,unit,Mamia Ultra-fit Peppa Pig Nappy Pants 22 Pack/...,2024-04-13,baby_products,False,mamia ultrafit peppa pig nappy pants size 6,mamia ultrafit peppa pig nappy pants size 6
1,Aldi,3.09,0.17,unit,Mamia Ultra-fit Peppa Pig Nappy Pants 18 Pack/...,2024-04-13,baby_products,False,mamia ultrafit peppa pig nappy pants size 7,mamia ultrafit nappy pants size 7
2,Aldi,3.59,0.09,unit,Mamia Ultra-fit Nappy Pants 40 Pack/Size 4,2024-04-13,baby_products,False,mamia ultrafit nappy pants size 4,mamia ultrafit nappy pants size 4
3,Aldi,4.79,0.32,unit,Mamia Boy's Night Pants 15 Pack,2024-04-13,baby_products,False,mamia boys night pants,mamia boys night pants
4,Aldi,4.79,0.32,unit,Mamia Girl's Night Pants 15 Pack,2024-04-13,baby_products,False,mamia girls night pants,mamia bed time bath
5,Aldi,0.85,0.04,unit,Mamia Newborn Nappies 24 Pack/Size 1 2-5kg/4-1...,2024-04-13,baby_products,False,mamia newborn nappies size 1 2411lbs,mamia eco nappies size 2 mini 3613lbs
6,Aldi,2.79,0.13,unit,Mamia Nappies Ultra Dry Air System 22 Pack/Siz...,2024-04-13,baby_products,False,mamia nappies ultra dry air system size 7 xxl,mamia nappies ultra dry air system size 7 xxl
7,Aldi,2.89,0.05,unit,Mamia Newborn Nappies 56 Pack/Size 3 Midi 4-9k...,2024-04-13,baby_products,False,mamia newborn nappies size 3 midi 4920lbs,mamia eco nappies size 2 mini 3613lbs
8,Aldi,2.25,0.04,unit,Mamia Newborn Nappies 60 Pack/Size 2 Mini 3-6k...,2024-04-13,baby_products,False,mamia newborn nappies size 2 mini 3613lbs,mamia newborn nappies size 2 mini 3613lbs
9,Aldi,3.59,0.13,unit,Mamia Ultra-fit Nappy Pants 28 Pack/Size 7,2024-04-13,baby_products,False,mamia ultrafit nappy pants size 7,mamia ultrafit nappy pants size 7


## The Payoff: Basket Level Analysis

In [15]:
sns.set_theme(style="whitegrid")
PROCESSED_DATA_PATH = "C:/Project/UK store analysis/data/02_processed/canonical_products_e5.parquet"
df_canonical = pd.read_parquet(PROCESSED_DATA_PATH)

print("Canonical product data loaded successfully.")

Canonical product data loaded successfully.


In [16]:
# Create the competitive Pivol Table
latest_date = df_canonical["date"].max()
df_latest = df_canonical[df_canonical["date"] == latest_date].copy()

# Drop duplicates in case we have multiple products on same data for a retailer
df_latest = df_latest.drop_duplicates(subset=["canonical_name", "supermarket"])

# Pivot on the new canonical_name column
price_pivot_canonical = df_latest.pivot_table(
    index = "canonical_name",
    columns="supermarket",
    values="prices"
)

print(f"Created a competitive pivot table with {price_pivot_canonical.shape[0]} unique canonical products")
price_pivot_canonical.head()

Created a competitive pivot table with 67341 unique canonical products


supermarket,ASDA,Aldi,Morrisons,Sains,Tesco
canonical_name,,,,,
0 fat greek style,NaN,NaN,1.45,NaN,NaN
0 fat greek style yogurt,0.85,NaN,1.10,NaN,1.95
0 fat greek style yogurt 4x100g,1.15,NaN,NaN,NaN,1.15
0 fat greek style yogurt strawberry,NaN,NaN,1.45,NaN,NaN
0 fat natural yogurt,1.10,NaN,1.09,NaN,1.70


In [17]:
# product_index = price_pivot_canonical.index.tolist()

# def find_products_by_keywords_filtered(keywords, product_index, max_items=15):
#     matches = []
#     for product in product_index:
#         # Skip if product name starts with a digit (likely pack size)
#         if product[0].isdigit():
#             continue
#         name = product.lower()
#         if any(keyword.lower() in name for keyword in keywords):
#             matches.append(product)
#         if len(matches) >= max_items:
#             break
#     return matches

# # Broader keywords for each basket
# essentials_keywords = [
#     "milk", "bread", "egg", "cheese", "potato", "butter", "bean", "banana",
#     "yogurt", "tomato", "onion", "apple"
# ]

# big_brand_keywords = [
#     "coca-cola", "heinz", "kelloggs", "walkers", "nescafe", "hovis"
# ]

# healthy_choice_keywords = [
#     "almond", "avocado", "pasta", "spinach", "chicken", "yogurt", "rice", "broccoli", "carrot"
# ]

# # Find filtered products
# essentials = find_products_by_keywords_filtered(essentials_keywords, product_index, max_items=15)
# big_brand = find_products_by_keywords_filtered(big_brand_keywords, product_index, max_items=15)
# healthy_choice = find_products_by_keywords_filtered(healthy_choice_keywords, product_index, max_items=15)

# print("Essentials basket:\n", essentials)
# print("\nBig Brand Shop basket:\n", big_brand)
# print("\nHealthy Choice basket:\n", healthy_choice)

# # Build baskets dict
# baskets = {
#     "The Essentials": essentials,
#     "The Big Brand Shop": big_brand,
#     "The Healthy Choice": healthy_choice,
# }


In [18]:
category_counts = df_canonical['category'].value_counts()
print(category_counts)

category
food_cupboard      2112803
health_products    1608050
fresh_food         1339306
home               1337377
drinks             1086120
household           544690
free-from           348226
frozen              332651
pets                296586
baby_products       290635
bakery              232798
Name: count, dtype: int64


In [19]:
product_index = set(price_pivot_canonical.index.tolist())  

# Function to filter product list by keywords and limit number of items
def find_products_by_keywords_filtered(keywords, product_list, max_items=15):
    matches = []
    for product in product_list:
        if product[0].isdigit():  # skip pack sizes
            continue
        name = product.lower()
        if any(keyword.lower() in name for keyword in keywords):
            matches.append(product)
        if len(matches) >= max_items:
            break
    return matches


essentials_keywords = [
    "milk", "bread", "egg", "cheese", "potato", "butter", "bean", "banana",
    "yogurt", "tomato", "onion", "apple"
]

big_brand_keywords = [
    "coca-cola", "heinz", "kelloggs", "walkers", "nescafe", "hovis"
]

healthy_choice_keywords = [
    "almond", "avocado", "pasta", "spinach", "chicken", "yogurt", "rice", "broccoli", "carrot"
]

essentials = find_products_by_keywords_filtered(essentials_keywords, product_index, max_items=15)
big_brand = find_products_by_keywords_filtered(big_brand_keywords, product_index, max_items=15)
healthy_choice = find_products_by_keywords_filtered(healthy_choice_keywords, product_index, max_items=15)

# Category-based baskets 

selected_categories = [
    "food_cupboard",
    "health_products",
    "fresh_food",
    "drinks",
    "household",
    "free-from",
    "frozen",
    "pets",
    "baby_products",
    "bakery"
]

baskets_from_categories = {}
max_products_per_category = 20

for cat in selected_categories:
    # Filter df_canonical to get products in category
    products_in_cat = df_canonical[df_canonical['category'] == cat]['canonical_name'].unique()
    filtered_products = [p for p in products_in_cat if p in product_index and not p[0].isdigit()]
    # Limit the number of products per category
    baskets_from_categories[cat] = filtered_products[:max_products_per_category]

# Combine all baskets into one dictionary 

baskets = {
    "The Essentials": essentials,
    "The Big Brand Shop": big_brand,
    "The Healthy Choice": healthy_choice,
}

# Add category-based baskets with nicer names
category_basket_names = {
    "food_cupboard": "Food Cupboard",
    "health_products": "Health Products",
    "fresh_food": "Fresh Food",
    "drinks": "Drinks",
    "household": "Household",
    "free-from": "Free From",
    "frozen": "Frozen Foods",
    "pets": "Pets",
    "baby_products": "Baby Products",
    "bakery": "Bakery",
}

for cat, products in baskets_from_categories.items():
    baskets[category_basket_names.get(cat, cat)] = products

# Print basket summaries 

for basket_name, products in baskets.items():
    print(f"Basket: {basket_name} - {len(products)} products")
    print(products)
    print()


Basket: The Essentials - 15 products
['butterkist cinema sweet popcorn 6x12g', 'finest rosanna pink onions', 'mighty ultimate barista oat milk alternative long life', 'extra special farmhouse wholemeal rye bread', 'just essentials by white bread', 'starbucks daily brew coffee with milk', 'robinsons mini apple blackcurrant on the go squash', 'heinz no added sugar or salt tomato ketchup', 'kingfisher coconut milk rich creamy', 'bfree artisan style multiseed sourdough bread', 'extra special mango apple ginger chutney', 'innocent kids smoothies cherries strawberries apples 4x150ml', 'good boy chewy chicken sweet potato wedges', 'warburtons gluten free white bread sourdough', 'ryvita thins rosemary sea salt flatbreads']

Basket: The Big Brand Shop - 15 products
['heinz mash ups mayocue mayonnaise bbq sauce', 'heinz no added sugar or salt tomato ketchup', 'heinz by nature tomato mozzarella pasta 12 months', 'kelloggs special k golden crunchy golden clusters breakfast cereal', 'heinz fruity a